# GWExPy TimeSeries 相互運用性 (Interop) チュートリアル

このノートブックでは、`gwexpy` の `TimeSeries` クラスに追加された新しい相互運用機能 (Interoperability Features) を紹介します。
`gwexpy` は、Pandas, Xarray, PyTorch, Astropy などの一般的なデータサイエンス・ライブラリとの間で、シームレスにデータを変換することができます。

In [1]:
import numpy as np
from astropy import units as u
from gwpy.time import LIGOTimeGPS
from gwexpy.timeseries import TimeSeries

# サンプルデータの作成
# 10秒間、100Hzの正弦波データを生成します
rate = 100 * u.Hz
dt = 1 / rate
t0 = LIGOTimeGPS(1234567890, 0)
duration = 10 * u.s
size = int(rate * duration)
times = np.arange(size) * dt.value
data = np.sin(2 * np.pi * 1.0 * times)  # 1Hz sine wave

ts = TimeSeries(data, t0=t0, dt=dt, unit="V", name="demo_signal")
print("Original TimeSeries:")
print(ts)

ModuleNotFoundError: No module named 'gwexpy'

## 1. Pandas との連携

`to_pandas()` メソッドを使うと、`TimeSeries` を `pandas.Series` に変換できます。
インデックスは `datetime` (UTC), `gps`, `seconds` (Unix timestamp) から選択可能です。

In [ ]:
try:
    # Pandas Series への変換 (デフォルトは datetime index)
    s_pd = ts.to_pandas(index="datetime")
    print("\n--- Converted to Pandas Series ---")
    print(s_pd.head())
    
    # Pandas Series から TimeSeries への復元
    ts_restored = TimeSeries.from_pandas(s_pd, unit="V")
    print("\n--- Restored TimeSeries from Pandas ---")
    print(ts_restored)
    
except ImportError:
    print("Pandas is not installed.")

## 2. Xarray との連携

`xarray` は多次元のラベル付き配列を扱う強力なライブラリです。`to_xarray()` でメタデータを保持したまま変換できます。

In [ ]:
try:
    # Xarray DataArray への変換
    da = ts.to_xarray()
    print("\n--- Converted to Xarray DataArray ---")
    print(da)
    
    # メタデータ (attrs) が保持されているか確認
    print("Attributes:", da.attrs)
    
    # 復元
    ts_x = TimeSeries.from_xarray(da)
    
except ImportError:
    print("Xarray is not installed.")

## 3. PyTorch との連携 (Deep Learning)

ディープラーニングの前処理として、`TimeSeries` を直接 `torch.Tensor` に変換できます。GPU転送も可能です。

In [ ]:
try:
    import torch
    
    # PyTorch Tensor への変換
    tensor = ts.to_torch(dtype=torch.float32)
    print("\n--- Converted to PyTorch Tensor ---")
    print(f"Tensor shape: {tensor.shape}, dtype: {tensor.dtype}")
    
    # Tensor からの復元 (t0, dt は別途指定が必要)
    ts_torch = TimeSeries.from_torch(tensor, t0=ts.t0, dt=ts.dt, unit="V")
    print("\n--- Restored from Torch ---")
    print(ts_torch)
    
except ImportError:
    print("PyTorch is not installed.")

## 4. Astropy との連携

天文学分野で標準的な `astropy.timeseries.TimeSeries` との相互変換もサポートしています。

In [ ]:
try:
    # Astropy TimeSeries への変換
    ap_ts = ts.to_astropy_timeseries()
    print("\n--- Converted to Astropy TimeSeries ---")
    print(ap_ts[:5])
    
    # 復元
    ts_astro = TimeSeries.from_astropy_timeseries(ap_ts)
    print("\n--- Restored from Astropy ---")
    print(ts_astro)

except ImportError:
    print("Astropy is not installed.")

## まとめ

これらの機能により、GWExPyはデータの「ハブ」として機能し、解析、機械学習、可視化などの多様なタスクに対して柔軟に対応できるようになりました。